In [2]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


class CoronarySmallDataset(Dataset):
    def __init__(self, dicom_dir, binary_dir, keypoint_dir, output_dir, transform=None):
        self.dicom_dir = dicom_dir
        self.binary_dir = binary_dir
        self.keypoint_dir = keypoint_dir
        self.output_dir = output_dir
        self.transform = transform
        self.input_files = os.listdir(binary_dir)

    def __len__(self):
        return len(self.input_files)

    def __getitem__(self, idx):
        SIZE = 256
        dicom_path = os.path.join(self.dicom_dir, self.input_files[idx])
        binary_path = os.path.join(self.binary_dir, self.input_files[idx])
        keypoint_path = os.path.join(self.keypoint_dir, self.input_files[idx])
        output_path = os.path.join(self.output_dir, self.input_files[idx])

        dicom = cv2.imread(dicom_path, cv2.IMREAD_UNCHANGED)
        dicom = cv2.cvtColor(dicom, cv2.COLOR_RGBA2RGB)
        dicom = cv2.resize(dicom, (SIZE, SIZE))

        binary = cv2.imread(binary_path, cv2.IMREAD_UNCHANGED)
        binary = cv2.resize(binary, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)

        keypoint = cv2.imread(keypoint_path, cv2.IMREAD_UNCHANGED)
        keypoint = cv2.resize(keypoint, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)

        binary = np.expand_dims(binary, axis=2)
        keypoint = np.expand_dims(keypoint, axis=2)
        input = np.concatenate((dicom, binary, keypoint), axis=2)

        output = cv2.imread(output_path, cv2.IMREAD_UNCHANGED)
        output = cv2.resize(output, (SIZE, SIZE), interpolation=cv2.INTER_NEAREST)
        
        if self.transform:
            input = self.transform(input)
            output = torch.tensor(output, dtype=torch.long)
        
        return input, output


transform = transforms.Compose([
    transforms.ToTensor()
])

train_dicom_dir = '../images/images_train_original/input_dicom'
train_binary_dir = '../images/images_train_original/input'
train_keypoint_dir = '../images/images_train_original/keypoints'
train_output_dir = '../images/images_train_original/output'

val_dicom_dir = '../images/images_val/input_dicom'
val_binary_dir = '../images/images_val/input'
val_keypoint_dir = '../images/images_val/keypoints'
val_output_dir = '../images/images_val/output'

test_dicom_dir = '../images/images_test/input_dicom'
test_binary_dir = '../images/images_test/input'
test_keypoint_dir = '../images/images_test/keypoints'
test_output_dir = '../images/images_test/output'

train_dataset = CoronarySmallDataset(train_dicom_dir, train_binary_dir, train_keypoint_dir, train_output_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_dicom_dir, val_binary_dir, val_keypoint_dir, val_output_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_dicom_dir, test_binary_dir, test_keypoint_dir, test_output_dir, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=False)


In [3]:
from converter_RGB import convert_int_to_RGB
from large_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

model.load_state_dict(torch.load('model_1.pth', map_location=device))
model.eval()

def show_image(type, image_name):
    dir = f"..\images\images_test\{type}"
    print(dir)
    if type == 'output':
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        # img = img[:, :, 2]
        print("Oryginalnie: ", np.unique(img))
        
        # print("Po resize: NEAREST\t  ", np.unique(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)))
        # cv2.imshow('INTER_NEAREST', cv2.resize(convert_int_to_RGB(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)), (512, 512)))
        # print("Po resize: NEAREST_EXACT  ", np.unique(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST_EXACT)))
        # cv2.imshow('INTER_NEAREST_EXACT', cv2.resize(convert_int_to_RGB(cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST_EXACT)), (512, 512)))
        
        print("Po resize:  ", np.unique(cv2.resize(img, (256, 256))))
        # print(img)
        img = convert_int_to_RGB(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    else:
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        img = img * 7
    # img = cv2.resize(img, (256, 256))
    # np.set_printoptions(threshold=np.inf)
    cv2.imshow(type, img)

def predict(image_name):
    dir = '..\images\images_test\input'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model.predict(img)
        pred = pred.squeeze().cpu().numpy()
    print("Z sieci:    ", np.unique(pred))

    pred_image = convert_int_to_RGB(pred)
    pred_image = cv2.resize(pred_image, (512, 512))
    pred_image = cv2.cvtColor(pred_image, cv2.COLOR_BGR2RGB)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\3151909764.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_1.pth', map_locati

In [18]:
# image_name = "131aedfhs6pnf1fvtvp49mhdb2fucqzo22_29.png"
# image_name = "131aedfhs6pnf1fvtvp49mld7mqexnz322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mia892s56cf22_28.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
image_name = "131aedfhs6pnf1fvtvp49mi8hml81goj22_37.png"

# treningowe
# image_name = "13c2ur549vohc0jat2e6y42xk11_31.png"
# image_name = "131aedfhs6pnf1fvtvp49jstof8iams322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mff2plx3rdw22_24.png"
show_image("input", image_name)
show_image("output", image_name)
predict(image_name)

..\images\images_test\input
..\images\images_test\output
Oryginalnie:  [ 0  5  6  7  8  9 10 11 13 15 16 17]
Po resize:   [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Z sieci:     [ 0  5  6  7  8  9 13 15 17 18]


In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score


def compute_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    correct_clear = 0
    total = 0
    total_clear = 0

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            correct += (preds == masks).sum().item()
            correct_clear += (preds[masks != 0] == masks[masks != 0]).sum().item()
            total += masks.numel()
            total_clear += masks[masks != 0].numel()

    accuracy = correct / total
    accuracy_clear = correct_clear / total_clear
    return accuracy, accuracy_clear

In [5]:
def compute_precision_recall_f1(model, data_loader, device, num_classes=28):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            all_preds.append(preds.view(-1).cpu().numpy())
            all_targets.append(masks.view(-1).cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    FN = np.zeros(num_classes)

    for i in range(num_classes):
        TP[i] = ((all_preds == i) & (all_targets == i)).sum()
        FP[i] = ((all_preds == i) & (all_targets != i)).sum()
        FN[i] = ((all_preds != i) & (all_targets == i)).sum()
    
    TP[0], FP[0], FN[0] = 0, 0, 0
    # print(TP, FP, FN)

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    # print(TP.sum(), FP.sum(), FN.sum())

    precision_weighted = TP.sum() / (TP.sum() + FP.sum())
    recall_weighted = TP.sum() / (TP.sum() + FN.sum())
    f1_weighted = 2 * precision_weighted * recall_weighted / (precision_weighted + recall_weighted)

    return precision, recall, f1, precision_weighted, recall_weighted, f1_weighted

In [6]:
# learning_rate = 2.4, dropout_rate = 0.2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_1.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\2882635488.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_1.pth', map_locati

Train Accuracy: 98.63%
Train Accuracy Clear: 74.20%

Validation Accuracy: 97.73%
Validation Accuracy Clear: 57.45%


In [7]:
model.load_state_dict(torch.load('model_1.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\892274786.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_1.pth', map_locatio

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 87.56%, Recall: 70.76%, F1: 78.27%
Class 2: Precision: 74.80%, Recall: 72.73%, F1: 73.75%
Class 3: Precision: 81.51%, Recall: 84.78%, F1: 83.11%
Class 4: Precision: 64.76%, Recall: 82.75%, F1: 72.66%
Class 5: Precision: 88.28%, Recall: 95.51%, F1: 91.75%
Class 6: Precision: 64.26%, Recall: 63.80%, F1: 64.03%
Class 7: Precision: 50.83%, Recall: 62.61%, F1: 56.10%
Class 8: Precision: 63.92%, Recall: 65.12%, F1: 64.51%
Class 9: Precision: 34.82%, Recall: 62.60%, F1: 44.75%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 61.51%, Recall: 63.76%, F1: 62.61%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 39.36%, Recall: 62.79%, F1: 48.39%
Class 16: Precision: 34.86%, Recall: 35.56%, F1: 35.21%
Class 17: Precision: 61.33%, Recall: 57.61%, F1: 59.41%
Class 18: Precisi

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [8]:
# learning_rate = 2.4, dropout_rate = 0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_2.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\990498041.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_2.pth', map_locatio

Train Accuracy: 98.55%
Train Accuracy Clear: 72.56%

Validation Accuracy: 97.72%
Validation Accuracy Clear: 57.33%


In [9]:
model.load_state_dict(torch.load('model_2.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1689784218.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_2.pth', map_locati

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 86.34%, Recall: 70.52%, F1: 77.64%
Class 2: Precision: 74.67%, Recall: 72.86%, F1: 73.76%
Class 3: Precision: 82.71%, Recall: 75.17%, F1: 78.76%
Class 4: Precision: 64.75%, Recall: 77.24%, F1: 70.45%
Class 5: Precision: 90.81%, Recall: 89.79%, F1: 90.29%
Class 6: Precision: 60.62%, Recall: 69.81%, F1: 64.89%
Class 7: Precision: 56.00%, Recall: 71.77%, F1: 62.91%
Class 8: Precision: 67.22%, Recall: 66.68%, F1: 66.95%
Class 9: Precision: 32.09%, Recall: 68.01%, F1: 43.60%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 62.42%, Recall: 58.57%, F1: 60.43%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 37.20%, Recall: 68.56%, F1: 48.23%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 61.35%, Recall: 62.01%, F1: 61.68%
Class 18: Precision:

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1128418758.py:33: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * precision * recall / (precision + recall)


In [10]:
# learning_rate = 2.4, dropout_rate = 0.4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_3.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\4058094024.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_3.pth', map_locati

Train Accuracy: 98.32%
Train Accuracy Clear: 68.00%

Validation Accuracy: 97.65%
Validation Accuracy Clear: 56.00%


In [11]:
model.load_state_dict(torch.load('model_3.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1572690559.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_3.pth', map_locati

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 86.44%, Recall: 71.59%, F1: 78.32%
Class 2: Precision: 75.15%, Recall: 77.58%, F1: 76.34%
Class 3: Precision: 81.17%, Recall: 86.46%, F1: 83.73%
Class 4: Precision: 62.49%, Recall: 82.10%, F1: 70.96%
Class 5: Precision: 87.22%, Recall: 95.04%, F1: 90.96%
Class 6: Precision: 61.03%, Recall: 65.86%, F1: 63.35%
Class 7: Precision: 50.06%, Recall: 71.61%, F1: 58.93%
Class 8: Precision: 70.39%, Recall: 60.22%, F1: 64.91%
Class 9: Precision: 31.41%, Recall: 57.90%, F1: 40.73%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 63.41%, Recall: 55.40%, F1: 59.13%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 33.69%, Recall: 66.50%, F1: 44.72%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 55.74%, Recall: 59.23%, F1: 57.43%
Class 18: Precision:

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1128418758.py:31: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_29416\1128418758.py:32: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
